In [35]:
import pandas as pd
import numpy as np
from scipy.stats import t
from statsmodels.stats.power import tt_ind_solve_power

# Посчитаем необходимый размер выборки для ratio-метрики

Ниже показан кейс, когда мы рассчитываем прогнозное количество наблюдений перед экспериментом. У нас есть исторический датасет (сделаем из генератора) и по нему можно посчитать дисперсию желаемой метрики. В этом примере посчитаем количество наблюдений для CTR: клики / сессии. Если вы считаете CTR как клики / показы, то суть не меняется. Для разнообразия добавим еще метрику Average Revenue Per Session

In [36]:
np.random.seed(1)
n = 100000

# каждая строка – уникальный пользователь с суммой кликов, выручкой и сессий за период 
df = pd.DataFrame({
    'session_cnt': np.random.randint(low = 1, high = 30, size = n),
    'revenue_amt': np.random.randint(low = 0, high = 2000, size = n),
    'click_cnt': np.random.randint(low = 1, high = 100, size = n),
    'variant': np.random.randint(low = 0, high = 2, size = n)
})
display(df)

,session_cnt,revenue_amt,click_cnt,variant
0,6,973,85,1
1,12,488,89,0
2,13,493,2,0
3,9,57,18,0
4,10,226,24,0
...,...,...,...,...
99995,17,756,40,1
99996,23,440,22,1
99997,19,595,20,0
99998,8,1914,65,1


### Сделаем функцию расчета дисперсии дельта-методом

In [37]:
def calculate_ratio_variance(numerator: float, denominator: float) -> float:
    numerator_mean = np.mean(numerator)
    numerator_var = np.var(numerator)
    denominator_mean = np.mean(denominator)
    denominator_var = np.var(denominator)
    
    cov = np.mean((numerator - numerator_mean) * (denominator - denominator_mean))

    var = (numerator_var / denominator_mean ** 2
           + denominator_var * numerator_mean ** 2 / denominator_mean ** 4
           - 2 * numerator_mean / denominator_mean ** 3 * cov)
    return var

## Pipeline

In [38]:
ALPHA = 0.01
POWER = 0.8

# Добавим псевдоэффект для числителя потом посчитаем необходимое количество наблюдений для его нахождения. 
# Обычно в рамках A/B мы хотим чтобы именно числитель менялся, а знаменатель оставался прежним. 
# Будем делать прогноз для 5% эффекта
np.random.seed(1)
df.loc[df.variant == 1,'click_cnt'] = df.loc[df.variant == 1,'click_cnt'] * 1.05
df.loc[df.variant == 1,'revenue_amt'] = df.loc[df.variant == 1,'revenue_amt'] * 1.05

# Параметры метрики
METRIC_PARAMS = {
    # TODO добавить параметры метрики
    "clicks_per_session": {"num": "click_cnt", "den": "session_cnt"},
    "revenue_per_session": {"num": "revenue_amt", "den": "session_cnt"},
}

# По каждой метрике посчитаем все интересующие нас статистики
# У нас тут одна метрика, но можете добавить свои параметры
res = pd.DataFrame()
for metric, param in METRIC_PARAMS.items():
    
    # дисперсия для ratio
    var = calculate_ratio_variance(
        numerator=df.loc[df.variant == 0, param['num']], 
        denominator=df.loc[df.variant == 0, param['den']]
    )
    
    # средние ratio
    rto_0 = np.sum(df.loc[df.variant == 0, param['num']]) / np.sum(df.loc[df.variant == 0, param['den']])
    rto_1 = np.sum(df.loc[df.variant == 1, param['num']]) / np.sum(df.loc[df.variant == 1, param['den']])
    
    delta = rto_1 - rto_0
    effect_size = abs(delta) / np.sqrt(var)

    to_insert = {
        "metric": metric,
        "effect_size": effect_size
    }
    
    res = res.append(to_insert, ignore_index=True)

# Считаем необходимое количество наблюдений по fixed horizon
res['n_need'] = [tt_ind_solve_power(row[0], alpha=ALPHA, power=POWER) for row in zip(res['effect_size'])]
display(res)

/var/folders/m7/p0h2nb855k5_n3tkckyhqdf40000gp/T/ipykernel_98790/3543577547.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(to_insert, ignore_index=True)
/var/folders/m7/p0h2nb855k5_n3tkckyhqdf40000gp/T/ipykernel_98790/3543577547.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(to_insert, ignore_index=True)


,metric,effect_size,n_need
0,clicks_per_session,0.051861,8686.268007
1,revenue_per_session,0.042571,12890.153549
